In [1]:
import warnings
import numpy as np
from tqdm.notebook import tqdm
from tabulate import tabulate
from lightsim2grid import LightSimBackend
from lightsim2grid.solver import SolverType
from lightsim2grid.securityAnalysis import SecurityAnalysisCPP  # lightsim2grid
from lightsim2grid.gridmodel import init
import time
import pandapower as pp  # pandapower
import pandapower.networks as pn  # grid cases
import plotly.graph_objects as go  # plotting
from superposition_theorem import State  # this study
import grid2op
from grid2op.Parameters import Parameters 
from grid2op.Chronics import ChangeNothing
import tempfile
import os

# ordered per number of branches
case_names = ["case14", "case118", "case_illinois200", "case300", 
              "case1354pegase",
              "case1888rte", "GBnetwork", "case3120sp", "case2848rte", "case2869pegase", 
              "case6495rte", "case6515rte", "case9241pegase"
             ]

In [2]:
def compute_extended_ST(case, nb_unit_acts=1):
    # technical details for creating a state
    with tempfile.TemporaryDirectory() as dir:
        grid_path = os.path.join(dir, "grid.json")
        pp.to_json(case, grid_path)
        param = Parameters()
        param.ENV_DC = True  # force the computation of the powerflow in DC mode
        param.MAX_LINE_STATUS_CHANGED = 99999
        param.MAX_SUB_CHANGED = 99999
        param.NO_OVERFLOW_DISCONNECTION = True
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            env = grid2op.make("blank",
                               chronics_class=ChangeNothing,
                               grid_path=grid_path,
                               test=True,
                               backend=LightSimBackend(),  # for speed to compute the initial information
                               param=param,
                               _add_to_name=f"{case.bus.shape[0]}_bus",
                               )
            env.change_parameters(param)
            env.change_forecast_parameters(param)

    obs = env.reset()
    state = State.from_grid2op_obs(obs,
                                   line_ids_disc_unary=tuple(range(env.n_line)),
                                   when_error="warns")
    
    nb_line_per_sub = np.zeros(env.n_sub)
    for sub_id in range(env.n_sub):
        nb_line_per_sub[sub_id] += (type(env).line_or_to_subid == sub_id).sum()
        nb_line_per_sub[sub_id] += (type(env).line_ex_to_subid == sub_id).sum()
        
    # takes action on the first two substation where it is possible todo make it "more random"
    sub_ids = np.where((env.sub_info >= 4) & (nb_line_per_sub >= 2))[0][:nb_unit_acts]  

    unit_acts = []
    for sub_id in sub_ids:
        un_act = state.get_emptyact()
        un_act.set_subid(sub_id)
        elems = un_act.get_elem_sub()
        # assign a powerline per nodes at least (todo add more "randomness")
        topo = {"lines_id" : [(l_id, lnum % 2 + 1) for lnum, l_id in enumerate(elems["lines_id"])]}
        # randomnly assign a bus to anything else
        for k in ["loads_id", "gens_id", "storages_id"]:
            if k not in elems:
                continue
            tmp_ = np.random.choice([1, 2], len(elems[k]))
            topo[k] = [(el, tmp) for el, tmp in zip(elems[k], tmp_)]
        un_act.set_bus(**topo)
        unit_acts.append(un_act)
    state.add_unary_actions_grid2op(obs, subs_actions_unary=unit_acts)
    
    _, total_time, nb_cont  = state.compute_flows_n1(subs_actions=unit_acts, line_ids=tuple(range(env.n_line)))
    return total_time, nb_cont

In [3]:
def compute_lightsim2grid(case):
    """compute the full security analysis using lightsim2grid"""
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        gridmodel = init(case)
    # perform the action
    # .XXX(blablabla)
    
    # start the initial computation
    V = gridmodel.dc_pf(1.04 * np.ones(case.bus.shape[0], dtype=np.complex128), 10, 1e-7)
    if not V.shape:
        # ac pf has diverged
        warnings.warn(f"Impossible to compute the security analysis for {case.bus.shape[0]}: divergence")
    
    # initial the model
    sec_analysis = SecurityAnalysisCPP(gridmodel)
    sec_analysis.change_solver(SolverType.KLUDC)
    for branch_id in range(len(gridmodel.get_lines()) + len(gridmodel.get_trafos())):
        sec_analysis.add_n1(branch_id)
    
    # now do the security analysis
    beg = time.perf_counter()
    sec_analysis.compute(V, 10, 1e-7)
    vs_sa = sec_analysis.get_voltages()
    mw_sa = sec_analysis.compute_power_flows()
    tot_time = time.perf_counter() - beg
    return tot_time, sec_analysis.nb_solved()
    

In [4]:
def compute_pandapower(case):
    pp.rundcpp(case)  # run initial powerflow
    
    nb_cont = 0
    tot_time = 0.
    # now do the security analysis
    for branch_id in range(case.line.shape[0]):
        beg = time.perf_counter()
        case.line["in_service"].iloc[branch_id] = False
        pp.rundcpp(case, check_connectivity=False)
        if case["_ppc"]["success"]:
            tot_time += time.perf_counter() - beg
            nb_cont += 1
        case.line["in_service"].iloc[branch_id] = True
        
    for branch_id in range(case.trafo.shape[0]):
        beg = time.perf_counter()
        case.trafo["in_service"].iloc[branch_id]= False
        pp.rundcpp(case, check_connectivity=False)
        if case["_ppc"]["success"]:
            tot_time += time.perf_counter() - beg
            nb_cont += 1
        case.trafo["in_service"].iloc[branch_id] = True
    return tot_time, nb_cont

In [5]:
res_table = []
res_per_cont = []
nb_branch = []
for case_nm in tqdm(case_names):
    this_row = [case_nm, None, None, None]  # total time
    this_row_per_cont = [case_nm, None, None, None]  # time for a single contingency
    
    # retrieve the case file from pandapower
    case = getattr(pn, case_nm)()
    nb_branch.append(case.line.shape[0] + case.trafo.shape[0])
    
    # use extended ST
    total_time, nb_cont = compute_extended_ST(case)
    this_row[1] = total_time
    this_row_per_cont[1] = total_time / nb_cont
    ##### end extended ST
    
    # use lightsim2grid
    total_time, nb_cont = compute_lightsim2grid(case)
    this_row[2] = total_time
    this_row_per_cont[2] = total_time / nb_cont
    ##### end lightsim2grid
    
    # use pandapower
    total_time, nb_cont = compute_pandapower(case)
    this_row[3] = total_time
    this_row_per_cont[3] = total_time / nb_cont
    ##### end pandapower
    
    res_table.append(this_row)
    res_per_cont.append(this_row_per_cont)

  0%|          | 0/13 [00:00<?, ?it/s]

/home/donnotben/Documents/topology_superposition_theorem/superposition_theorem/grid_state.py:62: UserWarning: Impossible to disconnect powerline 18: no feasible solution found.
  warnings.warn(f"Impossible to disconnect powerline {l_id}: no feasible solution found.")
/home/donnotben/.local/lib/python3.8/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:276: MatrixRankWarning: Matrix is exactly singular
  warn("Matrix is exactly singular", MatrixRankWarning)
/home/donnotben/Documents/topology_superposition_theorem/superposition_theorem/grid_state.py:62: UserWarning: Impossible to disconnect powerline 6: no feasible solution found.
  warnings.warn(f"Impossible to disconnect powerline {l_id}: no feasible solution found.")
/home/donnotben/Documents/topology_superposition_theorem/superposition_theorem/grid_state.py:62: UserWarning: Impossible to disconnect powerline 7: no feasible solution found.
  warnings.warn(f"Impossible to disconnect powerline {l_id}: no feasible solution found.")


In [6]:
print(tabulate(res_table, headers=["Grid", "Ext ST", "lightsim2grid", "pandapower"], floatfmt=".2e"))

Grid                Ext ST    lightsim2grid    pandapower
----------------  --------  ---------------  ------------
case14            1.35e-03         1.73e-04      1.49e-01
case118           1.85e-02         1.04e-02      1.57e+00
case_illinois200  2.30e-02         2.00e-02      2.37e+00
case300           4.78e-02         5.16e-02      3.93e+00
case1354pegase    4.96e-01         2.42e+00      3.06e+01
case1888rte       6.92e-01         2.81e+00      4.49e+01
GBnetwork         1.46e+00         5.58e+00      6.29e+01
case3120sp        2.08e+00         5.47e+00      8.72e+01
case2848rte       1.67e+00         9.99e+00      8.68e+01
case2869pegase    2.78e+00         1.55e+01      1.18e+02
case6495rte       3.74e+01         5.20e+01      3.73e+02
case6515rte       3.60e+01         5.07e+01      3.82e+02
case9241pegase    1.42e+02         4.06e+02      1.14e+03


In [7]:
print(tabulate(res_per_cont, headers=["Grid", "Ext ST", "lightsim2grid", "pandapower"], floatfmt=".2e"))

Grid                Ext ST    lightsim2grid    pandapower
----------------  --------  ---------------  ------------
case14            7.12e-05         9.13e-06      7.45e-03
case118           1.04e-04         5.88e-05      8.42e-03
case_illinois200  1.24e-04         1.15e-04      9.66e-03
case300           1.37e-04         1.60e-04      9.56e-03
case1354pegase    3.42e-04         1.69e-03      1.53e-02
case1888rte       4.25e-04         1.80e-03      1.77e-02
GBnetwork         5.52e-04         2.21e-03      1.96e-02
case3120sp        6.64e-04         1.85e-03      2.36e-02
case2848rte       6.60e-04         4.22e-03      2.30e-02
case2869pegase    7.25e-04         4.08e-03      2.59e-02
case6495rte       5.41e-03         8.04e-03      4.14e-02
case6515rte       5.22e-03         7.83e-03      4.23e-02
case9241pegase    9.69e-03         2.82e-02      7.09e-02


In [8]:
print(tabulate(res_table, headers=["Grid", "Ext ST", "lightsim2grid", "pandapower"], tablefmt="latex", floatfmt=".2e"))

\begin{tabular}{lrrr}
\hline
 Grid             &   Ext ST &   lightsim2grid &   pandapower \\
\hline
 case14           & 1.35e-03 &        1.73e-04 &     1.49e-01 \\
 case118          & 1.85e-02 &        1.04e-02 &     1.57e+00 \\
 case\_illinois200 & 2.30e-02 &        2.00e-02 &     2.37e+00 \\
 case300          & 4.78e-02 &        5.16e-02 &     3.93e+00 \\
 case1354pegase   & 4.96e-01 &        2.42e+00 &     3.06e+01 \\
 case1888rte      & 6.92e-01 &        2.81e+00 &     4.49e+01 \\
 GBnetwork        & 1.46e+00 &        5.58e+00 &     6.29e+01 \\
 case3120sp       & 2.08e+00 &        5.47e+00 &     8.72e+01 \\
 case2848rte      & 1.67e+00 &        9.99e+00 &     8.68e+01 \\
 case2869pegase   & 2.78e+00 &        1.55e+01 &     1.18e+02 \\
 case6495rte      & 3.74e+01 &        5.20e+01 &     3.73e+02 \\
 case6515rte      & 3.60e+01 &        5.07e+01 &     3.82e+02 \\
 case9241pegase   & 1.42e+02 &        4.06e+02 &     1.14e+03 \\
\hline
\end{tabular}


In [9]:
print(tabulate(res_per_cont, headers=["Grid", "Ext ST", "lightsim2grid", "pandapower"], tablefmt="latex", floatfmt=".2e"))

\begin{tabular}{lrrr}
\hline
 Grid             &   Ext ST &   lightsim2grid &   pandapower \\
\hline
 case14           & 7.12e-05 &        9.13e-06 &     7.45e-03 \\
 case118          & 1.04e-04 &        5.88e-05 &     8.42e-03 \\
 case\_illinois200 & 1.24e-04 &        1.15e-04 &     9.66e-03 \\
 case300          & 1.37e-04 &        1.60e-04 &     9.56e-03 \\
 case1354pegase   & 3.42e-04 &        1.69e-03 &     1.53e-02 \\
 case1888rte      & 4.25e-04 &        1.80e-03 &     1.77e-02 \\
 GBnetwork        & 5.52e-04 &        2.21e-03 &     1.96e-02 \\
 case3120sp       & 6.64e-04 &        1.85e-03 &     2.36e-02 \\
 case2848rte      & 6.60e-04 &        4.22e-03 &     2.30e-02 \\
 case2869pegase   & 7.25e-04 &        4.08e-03 &     2.59e-02 \\
 case6495rte      & 5.41e-03 &        8.04e-03 &     4.14e-02 \\
 case6515rte      & 5.22e-03 &        7.83e-03 &     4.23e-02 \\
 case9241pegase   & 9.69e-03 &        2.82e-02 &     7.09e-02 \\
\hline
\end{tabular}


In [10]:
fig = go.Figure()
fig.add_trace(go.Line(x=nb_branch, y=[el[1] for el in res_table], name="extended ST"))
fig.add_trace(go.Line(x=nb_branch, y=[el[2] for el in res_table], name="lightsim2grid"))
fig.add_trace(go.Line(x=nb_branch, y=[el[3] for el in res_table], name="pandapower"))
# fig.add_trace(go.Line(x=nb_branch, y=nb_branch))
fig.update_xaxes(title_text="Grid size (number of branch)")
fig.update_yaxes(title_text="Computation time (s) [log scale]")
fig.update_yaxes(type="log")
fig

/home/donnotben/.local/lib/python3.8/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [11]:
fig = go.Figure()
fig.add_trace(go.Line(x=nb_branch, y=[el[1] for el in res_per_cont], name="extended ST"))
fig.add_trace(go.Line(x=nb_branch, y=[el[2] for el in res_per_cont], name="lightsim2grid"))
fig.add_trace(go.Line(x=nb_branch, y=[el[3] for el in res_per_cont], name="pandapower"))
# fig.add_trace(go.Line(x=nb_branch, y=nb_branch, name="test"))
fig.update_xaxes(title_text="Grid size (number of branch)")
fig.update_yaxes(title_text="Computation time per cont. (s) [log scale]")
fig.update_yaxes(type="log")
fig

In [16]:
fig = go.Figure()
ext_st = np.array([el[1] for el in res_table])
fig.add_trace(go.Line(x=nb_branch, y=np.ones(len(res_per_cont)), name="vs extended ST"))
fig.add_trace(go.Line(x=nb_branch, y=np.array([el[2] for el in res_table]) / ext_st, name="vs lightsim2grid"))
fig.add_trace(go.Line(x=nb_branch, y=np.array([el[3] for el in res_table]) / ext_st, name="vs pandapower"))
# fig.add_trace(go.Line(x=nb_branch, y=nb_branch, name="test"))
fig.update_xaxes(title_text="Grid size (number of branch)")
fig.update_yaxes(title_text="Ext. ST speed up [log scale]")
fig.update_yaxes(type="log")
fig

/home/donnotben/.local/lib/python3.8/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


